In [440]:
from ortools.sat.python import cp_model
import random
from collections import defaultdict

In [441]:
from typing import List

class NursePatientMatcher:
    
    def __init__(self, num_patients, num_nurses, max_patients, patient_types, acuities, patient_times, hours_worked):
        self.p: int = num_patients
        self.n: int = num_nurses
        self.max_patients_per_nurse: int = max_patients
        self.patients_of_each_type: List[int] = patient_types
        self.patient_nurse_acuities: List[List[int]] = acuities
        self.patient_times: List[int] = patient_times
        self.hours_worked: int = hours_worked

In [442]:
def create_variables(self):
    model: cp_model.CpModel = self.model
    n: int = self.n
    p: int = self.p
    patients_per_types = self.patients_of_each_type
    max_patients: int = self.max_patients_per_nurse
    self.x = {}
    self.patients_per_nurse = {}
    self.patient_types = {}
    self.penalty = model.NewIntVar(0,1,'penalty')
    for nurse in range(n):
        for patient in range(p):
            self.x[nurse, patient] = model.NewBoolVar(f'x[Nurse: {nurse}, Patient: {patient}]')
    
    for nurse in range(n):
        self.patients_per_nurse[nurse] = model.NewIntVar(0, max_patients, f'Patients for Nurse {nurse}')
    
    prev = 0
    t = 0
    for patient in range(p):
        num_patients_of_type = patients_per_types[t]
        if patient >= prev + num_patients_of_type:
            prev = prev + num_patients_of_type
            t += 1
        self.patient_types[patient] = t

    self.patients_per_time = {}
    for i in range(self.hours_worked):
        self.patients_per_time[i] = [patient for patient in range(p) if self.patient_times[patient] == i]


    self.nurse_sched = []
    for nurse in range(n):
        self.nurse_sched.append([model.NewIntVar(0, p, f'Hour {i} for nurse {nurse}') for i in range(self.hours_worked)])
            
        

# Register this method with the solver class
NursePatientMatcher.create_variables = create_variables

In [443]:
def bound_patients_per_nurse(self):
    x = self.x
    model: cp_model.CpModel = self.model
    n: int = self.n
    p: int = self.p
    max_patients: int = self.max_patients_per_nurse
    for nurse in range(n):
        model.Add(sum(x[nurse,patient] for patient in range(p)) <= max_patients)

# Register this method with the solver class
NursePatientMatcher.bound_patients_per_nurse = bound_patients_per_nurse

In [444]:
def one_nurse_per_patient(self):
    x = self.x
    model: cp_model.CpModel = self.model
    n: int = self.n
    p: int = self.p
    for patient in range(p):
        model.Add(sum(x[nurse,patient] for nurse in range(n)) == 1)


# Register this method with the solver class
NursePatientMatcher.one_nurse_per_patient = one_nurse_per_patient

In [445]:
def fill_nurse_schedule(self):
    x = self.x
    model: cp_model.CpModel = self.model
    n: int = self.n
    p: int = self.p
    nurse_sched = self.nurse_sched
    patients_per_time = self.patients_per_time

    for nurse in range(n):
        for i in range(self.hours_worked):
            model.Add(nurse_sched[nurse][i] == sum(x[nurse, patient] for patient in patients_per_time[i]))
            model.Add(nurse_sched[nurse][i] <= 1)

# Register this method with the solver class
NursePatientMatcher.fill_nurse_schedule = fill_nurse_schedule
    

In [446]:
def track_patients_per_nurse(self):
    x = self.x
    model: cp_model.CpModel = self.model
    n: int = self.n
    p: int = self.p
    patients_per_nurse = self.patients_per_nurse
    for nurse in range(n):
        model.Add(sum(x[nurse,patient] for patient in range(p)) == patients_per_nurse[nurse])


# Register this method with the solver class
NursePatientMatcher.track_patients_per_nurse = track_patients_per_nurse

In [447]:
def at_most_one_double_shift(self):
    x = self.x
    model: cp_model.CpModel = self.model
    n: int = self.n
    nurse_sched = self.nurse_sched
    for nurse in range(n):
        num_double_tracker = [model.NewBoolVar(f'Double shifts for nurse {nurse}') for _ in range(self.hours_worked-1)]
        for i in range(1, self.hours_worked):
            model.Add((nurse_sched[nurse][i] + nurse_sched[nurse][i-1] == 2)).OnlyEnforceIf(num_double_tracker[i-1])
            model.Add((nurse_sched[nurse][i] + nurse_sched[nurse][i-1] < 2)).OnlyEnforceIf(num_double_tracker[i-1].Not())
        
        pairs = [[model.NewBoolVar(f'Shift pair {j}, {k} for nurse {nurse}') for k in range(j+1, self.hours_worked-1)] for j in range(self.hours_worked-1)]

        for j in range(self.hours_worked-1):
            for k in range(j+1, self.hours_worked-1):
                model.AddBoolOr([num_double_tracker[j].Not(), num_double_tracker[k].Not()]).OnlyEnforceIf(pairs[j][k-j-1])
                model.AddBoolAnd([num_double_tracker[j], num_double_tracker[k]]).OnlyEnforceIf(pairs[j][k-j-1].Not())

        flattened_pairs = []
        for pair in pairs:
            flattened_pairs.extend(pair)
        
        model.AddBoolAnd(flattened_pairs)
        
# Register this method with the solver class
NursePatientMatcher.at_most_one_double_shift = at_most_one_double_shift

In [448]:
def double_shift_penalty(self):
    x = self.x
    model: cp_model.CpModel = self.model
    n: int = self.n
    nurse_sched = self.nurse_sched
    self.penalty = model.NewIntVar(0, self.hours_worked*n, 'penalty')
    num_double_tracker = [[model.NewBoolVar(f'Double shifts for nurse {nurse}') for _ in range(self.hours_worked-1)] for nurse in range(n)]
    for nurse in range(n):
        for i in range(1, self.hours_worked):
            model.Add((nurse_sched[nurse][i] + nurse_sched[nurse][i-1] == 2)).OnlyEnforceIf(num_double_tracker[nurse][i-1])
            model.Add((nurse_sched[nurse][i] + nurse_sched[nurse][i-1] < 2)).OnlyEnforceIf(num_double_tracker[nurse][i-1].Not())
        
    model.Add(self.penalty == sum(sum(num_double_tracker[nurse][i] for i in range(self.hours_worked-1)) for nurse in range(n)))
        
# Register this method with the solver class
NursePatientMatcher.double_shift_penalty = double_shift_penalty

In [449]:
def minimize_objectives(self):
    x = self.x
    model: cp_model.CpModel = self.model
    patients_per_nurse = self.patients_per_nurse
    n: int = self.n
    p: int = self.p
    max_patients: int = self.max_patients_per_nurse
    acuities = self.patient_nurse_acuities
    penalty = self.penalty

    absolute_diff = [model.NewIntVar(0, max_patients, "Absolute difference between nurse n workload and average workload") for _ in range(n)]

    for nurse in range(n):
        difference = model.NewIntVar(-max_patients, max_patients, f'Difference between nurse {nurse} workload and average workload')
        model.Add(difference == patients_per_nurse[nurse] - int(p/n))
        model.AddAbsEquality(absolute_diff[nurse], difference)
    

    self.acuity_score = sum(acuities[nurse][self.patient_types[patient]] * self.x[nurse, patient] for nurse in range(n) for patient in range(p))

    acuity_ub = sum(max(acuities[nurse]) for nurse in range(n))
    acuity_lb = sum(min(acuities[nurse]) for nurse in range(n))

    M = acuity_ub - acuity_lb + 1

    model.Minimize(M*sum(absolute_diff) - self.acuity_score + M*penalty)
    

# Register this method with the solver class
NursePatientMatcher.minimize_objectives = minimize_objectives

In [450]:
def solve(self):
    self.model = cp_model.CpModel()
    self.solver = cp_model.CpSolver()
    self.create_variables()
    self.bound_patients_per_nurse()
    self.one_nurse_per_patient()
    self.fill_nurse_schedule()
    self.track_patients_per_nurse()
    self.double_shift_penalty()
    # self.at_most_one_double_shift()
    self.minimize_objectives()
    if self.solver.Solve(self.model) == cp_model.OPTIMAL:
        print('Solved!')
        print(f'Average Nurse Acuity Score: {self.solver.Value(self.acuity_score) / self.n}')
        for nurse in range(self.n):
            print(f'Patients for nurse {nurse}: {self.solver.Value(self.patients_per_nurse[nurse])}')
            for i in range(self.hours_worked):
                print(f'{self.solver.Value(self.nurse_sched[nurse][i])}', end=' ')
            print()
        
    else:   
        raise ValueError('Modeling error!')

# Register this method with the solver class
NursePatientMatcher.solve = solve

In [451]:
def read_data_and_solve(path, hours_worked, arrivals):
    with open(path) as f:
        lines = f.readlines()
        num_nurses, num_patients, num_patient_types = lines[0].split(' ')
        num_nurses, num_patients, num_patient_types = int(num_nurses), int(num_patients), int(num_patient_types)

        min_patients, max_patients = lines[1].split(' ')
        min_patients, max_patients = int(min_patients), int(max_patients)

        patient_types = lines[3].strip().split(' ')
        patient_types = [int(x) for x in patient_types]

        acuities = []
        for i in range(num_nurses):
            nurse_acuities = lines[5 + i].strip().split(' ')
            acuities.append([int(x) for x in nurse_acuities])
    
    patient_times = []
    if not arrivals:
        times_seen = defaultdict(int)
        for _ in range(num_patients):
            new_time = random.randint(0, hours_worked-1)
            while not (not times_seen or times_seen[new_time] < num_nurses):
                new_time = random.randint(0, hours_worked-1)
            times_seen[new_time] += 1
            patient_times.append(new_time)
    else:
        patient_times = arrivals

    print(patient_times)

    matcher = NursePatientMatcher(num_patients, num_nurses, max_patients, patient_types, acuities, patient_times, hours_worked)
    soln = matcher.solve()

In [452]:
read_data_and_solve('data/hospital_data.txt', 10, None)

[8, 6, 9, 0, 2, 4, 5, 5, 0, 3, 9, 8, 7, 0, 5, 6, 8, 2, 4, 1, 8, 7, 9, 4, 5, 0, 5, 8, 6, 2, 7, 4, 8, 4, 9, 2, 7, 5, 2, 3, 0, 1, 9, 4, 3, 4, 0, 9, 0, 9, 3, 5, 9, 6, 2, 3, 5, 7, 2, 6, 0, 2, 6, 0, 1, 3, 2, 4, 2, 5, 1, 8, 3, 1, 4, 1, 7, 3, 4, 0, 3, 3, 9, 5, 3, 5, 8, 5, 4, 4, 8, 0, 3, 1, 9, 5, 0, 5, 8, 4, 3, 4, 2, 3, 1, 9, 2, 5, 7, 7, 6, 2, 8, 6, 8, 9, 7, 7, 5, 6, 3, 0, 3, 5, 1, 4, 7, 7, 2, 9, 0, 1, 0, 6, 5, 5, 8, 6, 3, 2, 9, 7, 4, 5, 3, 8, 8, 0, 4, 2, 7, 8, 1, 9, 2, 5, 9, 4, 2, 5, 6, 2, 4, 0, 4, 4, 3, 8, 8, 6, 4, 6, 8, 8, 1, 8, 4, 2, 5, 2, 3, 3, 1, 5, 2, 1, 1, 7, 8, 1, 4, 7, 2, 0, 0, 9, 7, 5, 1, 9, 9, 9, 0, 6, 8, 1, 6, 4, 7, 3, 3, 8, 0, 0, 9, 6, 7, 6, 5, 2, 6, 6, 1, 0, 8, 4, 5, 4, 6, 8, 8, 9, 6, 9, 0, 0, 0, 9, 1, 8, 7, 8, 0, 7, 2, 0, 0, 3, 1, 5, 8, 9, 2, 2, 6, 2, 7, 3, 2, 5, 7, 7, 2, 3, 5, 7, 9, 8, 7, 3, 7, 2, 7, 7, 6, 3, 8, 9, 0, 8, 4, 3, 8, 1, 6, 1, 8, 5, 8, 9, 4, 9, 1, 1, 0, 9, 1, 4, 1, 6, 8, 3, 1, 1, 6, 0, 9, 1, 4, 5, 6, 2, 1, 3, 5, 2, 1, 4, 9, 1, 2, 7, 1, 4, 8, 2, 1, 0, 9, 6, 8, 8, 6, 

** STOP HERE UNTIL YOU READ RESULTS **

Only run the cell below if you would like to see an example of our at most one double shift hard constraint on a smaller sample data set!

In [453]:
def solve_hard(self):
    self.model = cp_model.CpModel()
    self.solver = cp_model.CpSolver()
    self.create_variables()
    self.bound_patients_per_nurse()
    self.one_nurse_per_patient()
    self.fill_nurse_schedule()
    self.track_patients_per_nurse()
    # self.double_shift_penalty()
    self.at_most_one_double_shift()
    self.minimize_objectives()
    if self.solver.Solve(self.model) == cp_model.OPTIMAL:
        print('Solved!')
        print(f'Average Nurse Acuity Score: {self.solver.Value(self.acuity_score) / self.n}')
        for nurse in range(self.n):
            print(f'Patients for nurse {nurse}: {self.solver.Value(self.patients_per_nurse[nurse])}')
            for i in range(self.hours_worked):
                print(f'{self.solver.Value(self.nurse_sched[nurse][i])}', end=' ')
            print()
        
    else:   
        raise ValueError('Modeling error!')

# Register this method with the solver class
NursePatientMatcher.solve = solve_hard
read_data_and_solve('data/5nurse5patientType9.txt', 12, [1, 5, 10, 9, 8, 8, 5, 2, 11, 4, 3, 0, 9, 3, 3, 5, 2, 0, 11, 1, 7, 6, 3, 5, 8, 1, 7, 2])
NursePatientMatcher.solve = solve

[1, 5, 10, 9, 8, 8, 5, 2, 11, 4, 3, 0, 9, 3, 3, 5, 2, 0, 11, 1, 7, 6, 3, 5, 8, 1, 7, 2]
Solved!
Average Nurse Acuity Score: 112.2
Patients for nurse 0: 6
1 1 0 1 0 1 0 1 0 1 0 0 
Patients for nurse 1: 7
1 0 1 1 0 1 0 1 0 1 0 1 
Patients for nurse 2: 5
0 1 0 1 0 1 0 0 1 0 1 0 
Patients for nurse 3: 5
0 0 1 1 0 1 0 0 1 0 0 1 
Patients for nurse 4: 5
0 1 1 0 1 0 1 0 1 0 0 0 
